<a href="https://colab.research.google.com/github/divtosz/handson-ml2/blob/master/NeMo_Voice_Swap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip uninstall transformers

Uninstalling transformers-3.5.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.6/dist-packages/transformers-3.5.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-3.5.0


In [1]:
!pip install transformers==3.5

In [2]:
BRANCH = 'v1.0.0b2'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

  Using cached https://files.pythonhosted.org/packages/40/96/9024a1c07bbe5e16bdcbcbd021b608e37b32df4301ae2090aad27c24ffe6/pytest_runner-5.3.0-py3-none-any.whl
     |████████████████████████████████| 1.3MB 10.4MB/s 
     |████████████████████████████████| 51kB 10.0MB/s 
     |████████████████████████████████| 2.0MB 46.2MB/s 
     |████████████████████████████████| 102kB 16.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 102kB 15.5MB/s 
     |████████████████████████████████| 563kB 52.5MB/s 
     |████████████████████████████████| 102kB 15.7MB/s 
     |████████████████████████████████| 163kB 59.6MB/s 
     |████████████████████████████████| 133kB 62.6MB/s 
     |████████████████████████████████| 747kB 47.6MB/s 
     |████████████████████████████████| 112kB 62.9MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for pystoi: filename=pystoi-0.3.3-py2.py3-none-any.whl size=7781 sha256=ce6d0765751102358c1e5b54560e6

In [ ]:
!pip install braceexpand
!pip install webdataset
!pip install frozendict

In [ ]:
!pip install g2p_en

In [ ]:
!pip install torch_stft

In [3]:
# Ignore pre-production warnings
import warnings
warnings.filterwarnings('ignore')
import nemo
# Import Speech Recognition collection
import nemo.collections.asr as nemo_asr
# Import Natural Language Processing collection
import nemo.collections.nlp as nemo_nlp
# Import Speech Synthesis collection
import nemo.collections.tts as nemo_tts
# IPython will be used to listen to the audio
import IPython

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


[NeMo W 2021-02-12 06:50:00 experimental:28] Module nemo.collections.asr.data.audio_to_text.AudioToCharDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-02-12 06:50:00 experimental:28] Module nemo.collections.asr.data.audio_to_text.AudioToBPEDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-02-12 06:50:00 experimental:28] Module nemo.collections.asr.data.audio_to_text.AudioLabelDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-02-12 06:50:00 experimental:28] Module nemo.collections.asr.data.audio_to_text.TarredAudioToTextDataset is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-02-12 06:50:00 experimental:28] Module nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset is experimental, not ready for production and is not fully supporte

In [4]:
# Download audio sample which we'll try
# This is a sample from LibriSpeech Dev Clean dataset - the model hasn't seen it before
Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# Listen to it
IPython.display.Audio(Audio_sample)

--2021-02-12 06:50:25--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.106.154
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.106.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K  --.-KB/s    in 0.04s   

2021-02-12 06:50:25 (6.42 MB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



In [5]:
# Speech Recognition model - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet15x5Base-En").cuda()
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained
(model_name='Punctuation_Capitalization_with_DistilBERT').cuda()
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="Tacotron2-22050Hz").cuda()
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="WaveGlow-22050Hz").cuda()

[NeMo I 2021-02-12 06:51:11 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet15x5Base-En.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b2/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo
[NeMo I 2021-02-12 06:51:15 common:399] Instantiating model from pre-trained checkpoint
[NeMo I 2021-02-12 06:51:15 features:241] PADDING: 16
[NeMo I 2021-02-12 06:51:15 features:258] STFT using torch
[NeMo I 2021-02-12 06:51:31 modelPT:257] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b2/QuartzNet15x5Base-En/00869f9c89b8393ca3de640e0c536bd2/QuartzNet15x5Base-En.nemo.
[NeMo I 2021-02-12 06:51:31 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemonlpmodels/versions/1.0.0a5/files/Punctuation_Capitalization_with_DistilBERT.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b2/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/P

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
[NeMo W 2021-02-12 06:51:41 modelPT:110] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-02-12 06:51:41 modelPT:117] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    ds_item: null
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_samples: -1
    batch_size: 64
    



[NeMo I 2021-02-12 06:51:50 modelPT:257] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b2/Punctuation_Capitalization_with_DistilBERT/6906a3705a7989aa9995aed680fb5d22/Punctuation_Capitalization_with_DistilBERT.nemo.
[NeMo I 2021-02-12 06:51:50 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemottsmodels/versions/1.0.0a5/files/Tacotron2-22050Hz.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b2/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo
[NeMo I 2021-02-12 06:51:54 common:399] Instantiating model from pre-trained checkpoint


[NeMo W 2021-02-12 06:51:55 tacotron2:80] Your config is using an old NeMo yaml configuration. Please ensure that the yaml matches the current version in the main branch for future compatibility.


[NeMo I 2021-02-12 06:51:55 features:241] PADDING: 16
[NeMo I 2021-02-12 06:51:55 features:254] STFT using conv
[NeMo I 2021-02-12 06:51:57 modelPT:257] Model Tacotron2Model was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b2/Tacotron2-22050Hz/59f6603364e11524c03c95920e7afaa0/Tacotron2-22050Hz.nemo.
[NeMo I 2021-02-12 06:51:57 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemottsmodels/versions/1.0.0a5/files/WaveGlow-22050Hz.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b2/WaveGlow-22050Hz/63a329dc3e8b44ec2e07cd4209eeab2a/WaveGlow-22050Hz.nemo
[NeMo I 2021-02-12 06:52:24 common:399] Instantiating model from pre-trained checkpoint
[NeMo I 2021-02-12 06:52:33 features:241] PADDING: 16
[NeMo I 2021-02-12 06:52:33 features:254] STFT using conv
[NeMo I 2021-02-12 06:52:38 modelPT:257] Model WaveGlowModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b2/WaveGlow-22050Hz/63a329dc3e8b44ec2e07cd4209eeab2a/WaveGlow-22050Hz.nemo.


In [6]:
# Convert our audio sample to text
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  raw_text = transcription

# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]
print(f'\nRaw recognized text: {raw_text}. \nText with capitalization and punctuation: {text}')

[NeMo I 2021-02-12 06:53:09 punctuation_capitalization_model:317] Using batch size 1 for inference
[NeMo I 2021-02-12 06:53:09 punctuation_capitalization_dataset:134] Max length: 27
[NeMo I 2021-02-12 06:53:09 data_preprocessing:294] Some stats of the lengths of the sequences:
[NeMo I 2021-02-12 06:53:09 data_preprocessing:300] Min: 27 |                  Max: 27 |                  Mean: 27.0 |                  Median: 27.0
[NeMo I 2021-02-12 06:53:09 data_preprocessing:302] 75 percentile: 27.00
[NeMo I 2021-02-12 06:53:09 data_preprocessing:303] 99 percentile: 27.00
[NeMo I 2021-02-12 06:53:09 punctuation_capitalization_dataset:165] 0 are longer than 27
[NeMo I 2021-02-12 06:53:09 punctuation_capitalization_dataset:168] *** Example ***
[NeMo I 2021-02-12 06:53:09 punctuation_capitalization_dataset:169] i: 0
[NeMo I 2021-02-12 06:53:09 punctuation_capitalization_dataset:170] subtokens: [CLS] well i don ' t wish to see it any more observed phoebe turning away her eyes it is certainly ver

In [7]:
# A helper function which combines Tacotron2 and WaveGlow to go directly from 
# text to audio
def text_to_audio(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').numpy()

In [8]:
# This is our original audio sample
IPython.display.Audio(Audio_sample)

In [9]:
# This is what was recognized by the ASR model
print(raw_text)

well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very like the old portrait


In [10]:
# This is how punctuation model changed it
print(text)

Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very like the old portrait.


In [11]:
# Without punctuation
IPython.display.Audio(text_to_audio(raw_text), rate=22050)

In [12]:
# Final result - with punctuation
IPython.display.Audio(text_to_audio(text), rate=22050)